In [1]:
import pandas as pd
import numpy as np
import preprocess_helper
import os
from collections import Counter

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import enchant
checker = enchant.Dict("en_US")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\11099\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


***
`FAA`, or `FAA.csv`, and `NASA`, or `nasa_abbr.csv`, contain two dictionaries of aviation acronyms used by FAA (Federal Aviation Administration) and NASA (National Aeronautics and Space Administration). Note that `CASA` dictionary is temporarily commented out due to the quality issue.

`top_thirty`, or `FAA Core Airports 2014.csv` contains the tracon codes of thirty busiest **(or most popular? exact statistic?)** airports in 2014. We currently limit out analysis to these thirty airports. 

We are using the processed ASRS data by calling `load_asrs()` function in `preprocess_helper.py`. See line 173 in the [repo](https://github.com/Oskilab/aviation_integrated/blob/master/asrs_analysis/preprocess_helper.py). 

In [2]:
# Dictionaries of acronym
# CASA = pd.read_csv('dictionaries/CASA.csv')
FAA = pd.read_csv('dictionaries/FAA.csv')
NASA = pd.read_csv('dictionaries/nasa_abbr.csv')
top_thirty = pd.read_csv('dictionaries/FAA Core Airports 2014.csv')

# Limit to top thirty airports
ASRS = preprocess_helper.load_asrs(load_saved=True)
top_thirty_bool = ASRS['tracon_code'].isin(top_thirty['tracon_key'])
ASRS_30 = ASRS[top_thirty_bool]

# Accident and incident counts of FAA and NTSB
acc_inc = pd.read_csv(r"E:\aviation_integrated\results\combined_vol_incident.csv")

D:\Anaconda\envs\aviation_integrated\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (8,9,11,12,14,15,19,20,28,29,30,31,32,33,34,38,39,40,41,42,46,50,53,54,56,57,58,59,60,64,65,70,71,72,74,75,80,81,82,86,88,93,94,95,104) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


***
Here are some utility functions that help us count acronyms in the reports of ASRS in different ways. 


In [3]:
# Utility functions used for counting the acronyms in ASRS data

# Check if a word is spelled correctly (OBSOLETE)
# def spell_check(word):
#     return checker.check(word)
 
def count_acronyms(text, acronym):
    """
    Compute the frequency of all acronyms shown and the number of unique acronyms shown
    in the text paragraph according to a specified dictionary.
    
    Input arguments:
    TEXT - String. The text pargraph in which we find and count the acronyms.
    ACRONYMS - List of strings. Each entry is an acronym. 
    
    Returns:
    TOTAL - integer, frequency of all the acronyms shown in TEXT.
    UNIQUE - integer, the number of unique acronyms shown in TEXT
    """   
    # assert correct types of input arguments
    
    # create a dictionary containing the counts of distinct words
    all_words = text.lower().split()
    all_words = [x.strip('{[(,:.') for x in all_words]
    words_count = Counter(all_words)
    
    text_acronym = [word for word in words_count.keys() if word in acronym]
    unique = len(text_acronym)
    total = np.sum([words_count[word] for word in text_acronym])   
        
    return total, unique

# Testing count_acronyms
sample_text = "Go bears, we bears will beat stanford oh yeah let's go."
acronyms = ['go', 'bears', 'bear']

assert count_acronyms(sample_text, acronyms) == (4, 2)

***
We clean the dictionaries here. Note that `CASA` dictionary is temporarily commented out due to the quality issue.

We remove the stop words, such as *me*, *she*, and *am*, from the acronyms to avoid confusion. Suppose *me* is an acronym, then how can we know whether *me* in the text represents a stop word or an acronym?

Note that we temporarily discard removing the spelling check. By manual examination of the text, we notice that most acronyms that pass the spelling check are not used as normal English words in the text.

In [4]:
# Clean the field of acronyms

# CASA['acronym'] = CASA['acronym'].str.strip().str.lower()
# CASA['acronym'] = CASA['acronym'].str.replace(r'\(.+\)', '', regex=True)
# CASA.drop_duplicates(["acronym"], inplace=True, keep='first')
# CASA_cleaned = CASA[['acronym']]

FAA['acronym'] = FAA['acronym'].str.strip().str.lower()
FAA.drop_duplicates(["acronym"], inplace=True, keep='first')
FAA_cleaned = FAA[['acronym']]

NASA['acronym'] = NASA['acronym'].str.strip().str.lower()
NASA.drop_duplicates(["acronym"], inplace=True, keep='first')
NASA_cleaned = NASA[['acronym']]

top_thirty['tracon_key'] = top_thirty['tracon_key'].str.strip().str.upper()
top_thirty.drop_duplicates(["tracon_key"], inplace=True, keep='first')
top_thirty_cleaned = top_thirty[['tracon_key']]

# Remove stop words in the dictionaries
stop_words = set(stopwords.words('english'))

# bool_index = CASA_cleaned.isin(stop_words)['acronym'].values
# CASA_cleaned = CASA_cleaned[~bool_index]

bool_index = FAA_cleaned.isin(stop_words)['acronym'].values
FAA_cleaned = FAA_cleaned[~bool_index]

bool_index = NASA_cleaned.isin(stop_words)['acronym'].values
NASA_cleaned = NASA_cleaned[~bool_index]

# Filter out the words passing the spelling check (OBSOLETE)
# bool_index = np.array(list(map(spell_check, list(CASA_cleaned['acronym'].values))))
# CASA_cleaned = CASA_cleaned[~bool_index]

# bool_index = np.array(list(map(spell_check, list(FAA_cleaned['acronym'].values))))
# FAA_cleaned = FAA_cleaned[~bool_index]

# bool_index = np.array(list(map(spell_check, list(NASA_cleaned['acronym'].values))))
# NASA_cleaned = NASA_cleaned[~bool_index]

***
## Create Master Key

`master_key` contains all possible combinations of `tracon` (top thirty airports), `year` (1988 - 2019), and `month` (January to December). It serves as a standard and exhausted key which helps us find the indices of `(tracon, year, month)`  that do not have any observation in the data

In [5]:
date = pd.date_range(start = '1/1/1988', end = '12/31/2019', freq = 'M')
date_df = pd.DataFrame(data = {'year': date.year, 'month': date.month})
master_key = top_thirty.merge(date_df, how = 'cross')
master_key = master_key.drop('ATADS_Type', axis = 1)
master_key = master_key.rename({'tracon_key': 'tracon'}, axis = 1)
master_key

,tracon,year,month
0,ATL,1988,1
1,ATL,1988,2
2,ATL,1988,3
3,ATL,1988,4
4,ATL,1988,5
...,...,...,...
11515,TPA,2019,8
11516,TPA,2019,9
11517,TPA,2019,10
11518,TPA,2019,11


In [6]:
master_key.to_csv("quality_check/master_key.csv")

***
## Merge FAA/NTSB with master key

We merge the data of accidents and incidents from FAA and NTSB with `master_key` we created above. We can easily see the advantage of using `master_key` to find the indices that have no observations. 

In [7]:
# Limit NTSB/FAA to top 30
acc_inc_bool = acc_inc['airport_code'].isin(top_thirty['tracon_key'])
acc_inc_30 = acc_inc.loc[acc_inc_bool]
acc_inc_30 = acc_inc_30.drop("Unnamed: 0", axis = 1)
acc_inc_30 = acc_inc_30.rename({'airport_code': 'tracon'}, axis = 1)
acc_inc_30.to_csv('quality_check/acc_inc_30.csv')

In [8]:
faa_ntsb_merge = master_key.merge(acc_inc_30, left_on=['tracon', 'year', 'month'], right_on=['tracon', 'year', 'month'], how='left')
faa_ntsb_merge.to_csv('quality_check/merge_acc_inc.csv')
faa_ntsb_merge

,tracon,year,month,ntsb_incidents,ntsb_accidents,faa_incidents,faa_ntsb_overlap,NTSB_FAA_incidents_total,NTSB_FAA_incidents_total_nodups
0,ATL,1988,1,0.0,0.0,1.0,0.0,1.0,1.0
1,ATL,1988,2,NaN,NaN,NaN,NaN,NaN,NaN
2,ATL,1988,3,1.0,0.0,4.0,1.0,5.0,4.0
3,ATL,1988,4,1.0,0.0,0.0,0.0,1.0,1.0
4,ATL,1988,5,0.0,2.0,0.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
11515,TPA,2019,8,0.0,1.0,0.0,0.0,1.0,1.0
11516,TPA,2019,9,0.0,0.0,2.0,0.0,2.0,2.0
11517,TPA,2019,10,NaN,NaN,NaN,NaN,NaN,NaN
11518,TPA,2019,11,NaN,NaN,NaN,NaN,NaN,NaN


**Note**: Perhaps use an indicator column that marks the index the missing data with ?

In [9]:
faa_ntsb_merge = faa_ntsb_merge.fillna(0)

***
## Moving average of ASRS

Here we utilize the idea of *moving average* to create aggregated statistics of acronym counts in ASRS data. The goal is to study the relationship between the acronym counts of past ASRS reports and the future incident and accident counts.

We first group all the observations in ASRS by `(tracon, year, month)`.



In [10]:
# collapse ASRS by [year, month, tracon_code]
asrs_collapse = ASRS_30.groupby(['tracon_code', 'year', 'month'])[['narrative']].agg(lambda x: x.str.cat(sep=' ')).reset_index()
asrs_collapse['num_obs'] = ASRS_30.groupby(['year', 'month', 'tracon_code']).size().values
asrs_collapse['AtcAdvisoryMultCount'] = ASRS_30.groupby(['year', 'month', 'tracon_code'])['AtcAdvisoryMultCount'].sum().values

In [11]:
asrs_collapse.head()

,tracon_code,year,month,narrative,num_obs,AtcAdvisoryMultCount
0,ATL,1988.0,1.0,apching the atl area a solid line of thunderst...,3,3
1,ATL,1988.0,2.0,established on ils rwy 27l atl on a 12 mile fi...,5,5
2,ATL,1988.0,3.0,f o was flying the acft and assigned a 060 deg...,1,1
3,ATL,1988.0,4.0,tca was inadvertently entered at 11500 in uppe...,5,6
4,ATL,1988.0,6.0,while being vectored for apch to rwy 9r atc di...,7,8


Then we merge the aggregated ASRS data with master key to find the indices that do not have observations and fill in the missing values corresponding to these indices.

In [12]:
# merge ASRS with master key
asrs_merge = master_key.merge(asrs_collapse, left_on=['tracon', 'year', 'month'], right_on=['tracon_code', 'year', 'month'], how='left')
asrs_merge = asrs_merge.loc[:, ['year', 'month', 'tracon', 'narrative', 'AtcAdvisoryMultCount', 'num_obs']]
asrs_merge['narrative'] = asrs_merge['narrative'].fillna('')
asrs_merge = asrs_merge.fillna(0)
asrs_merge.head(10)

,year,month,tracon,narrative,AtcAdvisoryMultCount,num_obs
0,1988,1,ATL,apching the atl area a solid line of thunderst...,3.0,3.0
1,1988,2,ATL,established on ils rwy 27l atl on a 12 mile fi...,5.0,5.0
2,1988,3,ATL,f o was flying the acft and assigned a 060 deg...,1.0,1.0
3,1988,4,ATL,tca was inadvertently entered at 11500 in uppe...,6.0,5.0
4,1988,5,ATL,,0.0,0.0
5,1988,6,ATL,while being vectored for apch to rwy 9r atc di...,8.0,7.0
6,1988,7,ATL,during enrte portion of flt we were assigned a...,6.0,5.0
7,1988,8,ATL,at about 200 agl on final apch to rwy 27l at a...,3.0,2.0
8,1988,9,ATL,mlg x was dsnded to 12000 mlg x read back 1000...,5.0,3.0
9,1988,10,ATL,at 13000 level flt 250 kts i asked the f o to ...,1.0,1.0


The next code cell is a key part. We first split the ASRS and FAA/NTSB data by `tracon`. We presume that the past reports from a certain airport will only affect the future accident and incidents of the same airport. **Possible to affect other airports as well?**

Then, we use a moving window with customized length (`window_len`) to scan over the ASRS data and create different groups of data. The stride is 1 (**customize it?**).  In each group, we concatenate all the text reports to a single pargraph and then count the acronyms in the paragraph. For each count statistic, such as `FAA_total`, we create the sum and average of it of all the observations in the group. The average will be per observation. **Do we need the average of observations?**

If a group does not have the number of observations which is equal to `window_len`, then we use 0 for all the aggregated count statistics. 

Finally, we merge all the aggregated count statistics with FAA/NTSB incident and accident data with customized offset or lag (`lag`). We expect the effect of past ASRS reports will not be immediately shown by the future FAA/NTSB incident and accident counts.

In [19]:
window_len = 6
lag = 1

# split the data by the tracon code
asrs_splits = [asrs_merge[asrs_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
faa_ntsb_splits = [faa_ntsb_merge[faa_ntsb_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
final = []

for i in range(len(asrs_splits)):
    asrs = asrs_splits[i].reset_index().drop('index', axis = 1)
    faa_ntsb = faa_ntsb_splits[i].reset_index().drop('index', axis = 1)
    
    # compute moving average of ASRS 
    windows = asrs.rolling(window = window_len, min_periods=window_len)
    num_obs_sum = []
    #num_obs_avg = []
    num_word_sum = []
    num_word_avg = []
    num_unique_word_sum = []
    num_unique_word_avg = []
    FAA_total_sum = []
    FAA_total_avg = []
    FAA_unique_sum = []
    FAA_unique_avg = []
    NASA_total_sum = []
    NASA_total_avg = []
    NASA_unique_sum = []
    NASA_unique_avg = []
    AtcAdvisoryMultCount_sum = []
    AtcAdvisoryMultCount_avg = []


    for window in windows:
        size = window.shape[0]

        # create a dictionary of counts of words
        text_concate = window['narrative'].str.cat(sep=' ')
        
        # number of observations
        num_obs = window['num_obs'].sum()
        num_obs_sum.append(num_obs)
        # num_obs_avg.append(num_obs / size)
        
        # number of words
        num_word = len(text_concate.split())
        num_word_sum.append(num_word)
        num_word_avg.append(num_word / num_obs)

        # number of unique words
        num_unique_word = len(set(text_concate.split()))
        num_unique_word_sum.append(num_unique_word)
        num_unique_word_avg.append(num_unique_word / num_obs)

        # number of FAA acronyms
        FAA_total, FAA_unique = count_acronyms(text_concate, FAA_cleaned['acronym'].values)
        FAA_total_sum.append(FAA_total)
        FAA_total_avg.append(FAA_total / num_obs)
        FAA_unique_sum.append(FAA_unique)
        FAA_unique_avg.append(FAA_unique / num_obs)

        # number of NASA acronyms
        NASA_total, NASA_unique = count_acronyms(text_concate, NASA_cleaned['acronym'].values)
        NASA_total_sum.append(NASA_total)
        NASA_total_avg.append(NASA_total / num_obs)
        NASA_unique_sum.append(NASA_unique)
        NASA_unique_avg.append(NASA_unique / num_obs)

        # number of atcadvisory splits
        num_splits = window['AtcAdvisoryMultCount'].sum()
        AtcAdvisoryMultCount_sum.append(num_splits)
        AtcAdvisoryMultCount_avg.append(num_splits / num_obs)
    
    # convert list to numpy arrays
    num_obs_sum = np.array(num_obs_sum)
    #num_obs_avg = np.array(num_obs_avg)
    num_word_sum = np.array(num_word_sum)
    num_word_avg =  np.array(num_word_avg)
    num_unique_word_sum = np.array(num_unique_word_sum)
    num_unique_word_avg = np.array(num_unique_word_avg)
    FAA_total_sum = np.array(FAA_total_sum)
    FAA_total_avg = np.array(FAA_total_avg)
    FAA_unique_sum = np.array(FAA_unique_sum)
    FAA_unique_avg = np.array(FAA_unique_avg)
    NASA_total_sum = np.array(NASA_total_sum)
    NASA_total_avg = np.array(NASA_total_avg)
    NASA_unique_sum = np.array(NASA_unique_sum)
    NASA_unique_avg = np.array(NASA_unique_avg)
    AtcAdvisoryMultCount_sum = np.array(AtcAdvisoryMultCount_sum)
    AtcAdvisoryMultCount_avg = np.array(AtcAdvisoryMultCount_avg)
      
    # add the counts to the dataframe
    asrs_moving = asrs.drop(['narrative', 'AtcAdvisoryMultCount'], axis = 1)
    asrs_moving['num_obs_sum'] = num_obs_sum
    #asrs_moving['num_obs_avg'] = num_obs_avg
    asrs_moving['num_word_sum'] = num_word_sum
    asrs_moving['num_word_avg'] = num_word_avg
    asrs_moving['num_unique_word_sum'] = num_unique_word_sum 
    asrs_moving['num_unique_word_avg'] = num_unique_word_avg 
    asrs_moving['FAA_total_sum'] = FAA_total_sum
    asrs_moving['FAA_total_avg'] = FAA_total_avg
    asrs_moving['FAA_unique_sum'] = FAA_unique_sum
    asrs_moving['FAA_unique_avg'] = FAA_unique_avg
    asrs_moving['NASA_total_sum'] = NASA_total_sum
    asrs_moving['NASA_total_avg'] = NASA_total_avg
    asrs_moving['NASA_unique_sum'] = NASA_unique_sum
    asrs_moving['NASA_unique_avg'] = NASA_unique_avg
    asrs_moving['AtcAdvisoryMultCount_sum'] = AtcAdvisoryMultCount_sum
    asrs_moving['AtcAdvisoryMultCount_avg'] = AtcAdvisoryMultCount_avg
    
    # insert empty rows at the beginning for lagging
    empty_row_df = pd.DataFrame(data = [pd.Series(index=asrs.columns) for _ in range(lag)])
    empty_row_df = empty_row_df.drop('tracon', axis = 1)   
    
    # merge the moving averages of ASRS with FAA+NTSB 
    asrs_moving = pd.concat([empty_row_df, asrs_moving])
    asrs_moving = asrs_moving.drop(['year', 'month', 'tracon'], axis = 1)
    asrs_moving = asrs_moving.reset_index().drop('index', axis = 1)
    merge_df = pd.concat([faa_ntsb, asrs_moving], axis = 1)
    final.append(merge_df)

final_df = pd.concat(final).reset_index().drop(['index', 'AtcAdvisoryMultCount', 'narrative', 'num_obs'], axis = 1)
final_df.insert(9, "num_obs", asrs_merge['num_obs'])
final_df.to_csv('quality_check/final.csv')

D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:111: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:47: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:52: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:57: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:59: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:64: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:66: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/b

In [20]:
final_df

,tracon,year,month,ntsb_incidents,ntsb_accidents,faa_incidents,faa_ntsb_overlap,NTSB_FAA_incidents_total,NTSB_FAA_incidents_total_nodups,num_obs,...,FAA_total_sum,FAA_total_avg,FAA_unique_sum,FAA_unique_avg,NASA_total_sum,NASA_total_avg,NASA_unique_sum,NASA_unique_avg,AtcAdvisoryMultCount_sum,AtcAdvisoryMultCount_avg
0,ATL,1988.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ATL,1988.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,32.0,10.666667,7.0,2.333333,134.0,44.666667,46.0,15.333333,3.0,1.000000
2,ATL,1988.0,3.0,1.0,0.0,4.0,1.0,5.0,4.0,1.0,...,106.0,13.250000,19.0,2.375000,334.0,41.750000,84.0,10.500000,8.0,1.000000
3,ATL,1988.0,4.0,1.0,0.0,0.0,0.0,1.0,1.0,5.0,...,136.0,15.111111,19.0,2.111111,475.0,52.777778,96.0,10.666667,9.0,1.000000
4,ATL,1988.0,5.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,...,140.0,10.000000,21.0,1.500000,481.0,34.357143,99.0,7.071429,15.0,1.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11545,TPA,2019.0,9.0,0.0,0.0,2.0,0.0,2.0,2.0,NaN,...,1.0,0.500000,1.0,0.500000,0.0,0.000000,0.0,0.000000,2.0,1.000000
11546,TPA,2019.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,1.0,0.500000,1.0,0.500000,0.0,0.000000,0.0,0.000000,2.0,1.000000
11547,TPA,2019.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,1.0,0.500000,1.0,0.500000,0.0,0.000000,0.0,0.000000,2.0,1.000000
11548,TPA,2019.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,1.0,0.500000,1.0,0.500000,0.0,0.000000,0.0,0.000000,2.0,1.000000


In [17]:
asrs_merge.to_csv("quality_check/asrs_merge.csv")
asrs_moving.to_csv("quality_check/asrs_moving.csv")

# Quality check

In [ ]:
try:
    os.mkdir("quality_check")
except OSError as error:
    print(error)

np.random.seed(8)   
final_merge.sample(10).to_csv("quality_check/final_merge_10.csv")

In [54]:
ASRS_30.to_csv('quality_check/asrs_30.csv')

In [15]:
ASRS_30.loc[(ASRS_30['tracon_code'] == 'ATL') & (ASRS_30['year'] == 1988) & (ASRS_30['month'] == 3)]['narrative']

739     f o was flying the acft and assigned a 060 deg...
765     on taxi in at alt sbnd on ramp 3 right side no...
768     mlg x landed rwy 9r and was told to hold short...
811     under clear weather conditions landed rwy 9r a...
848     the f o was lndg on rwy 27l at atl on 5 mi fin...
883     taxiout was normal on tkof felt acft vibration...
1051    atlanta apch ctlr was saturated radio calls we...
Name: narrative, dtype: object

In [138]:
test = ['mic', 'ctlr', 'hdg', 'capt', 'flt', 'abc', 'left', 'bbc', 'n', 'abeam']
for x in test:
    print(x in FAA_cleaned['acronym'].values)

False
False
False
False
False
False
False
False
False
False
